In [1]:
import pandas as pd
import utils
import numpy as np
from tqdm import tqdm
import collections
import time
import copy
import os
import datetime
from datetime import date
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
import smtplib
home = str(Path.home())
t1 = time.time()
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Snowflake connector dbt_jferragut
engine = utils.create_snowflake_engine(TAG = '')
connection = engine.connect()

In [3]:
base_data_sql = """
SELECT *
FROM test_hbg.dbt_jferragut.FEATURE_ENCODED"""

base_data = pd.read_sql_query(base_data_sql,engine)

In [5]:
profile = ProfileReport(base_data, title="Profiling Report")

In [ ]:
profile.to_file("EDA.pdf")

In [150]:
# EDA

In [ ]:
base_data.columns

In [ ]:
# numerical
data_0 = base_data[(base_data['is_fraud'] == 0)]['dicrepancy_destcat']
data_1 = base_data[base_data['is_fraud'] == 1]['dicrepancy_destcat']

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=False)  # sharey=False para escalas independientes

# Histograma para variable_binaria == 0
axes[0].hist(data_0,  bins=np.arange(-0.5, max(data_0)+1.5,1), color='blue', alpha=0.7, edgecolor='black')
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_xlabel('dicrepancy_destcat', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Histograma para variable_binaria == 1
axes[1].hist(data_1,  bins=np.arange(-0.5, max(data_0)+1.5,1), color='orange', alpha=0.7, edgecolor='black')
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xlabel('dicrepancy_destcat', fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

y_limit = [-1.5,30]
# x_limit = [0,5]
axes[1].set_xlim(y_limit)
axes[0].set_xlim(y_limit)
# axes[1].set_xlim(x_limit)
# # x_limit = [0,500]

# Ajustar espaciado entre los subplots
plt.tight_layout()
plt.show()

In [ ]:
# numerical relative
data_0 = base_data[(base_data['is_fraud'] == 0)]['dicrepancy_destcat']
data_1 = base_data[base_data['is_fraud'] == 1]['dicrepancy_destcat']

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=False)  # sharey=False para escalas independientes
#  bins=np.arange(0, max(data_0)+300,300)
#  bins=np.arange(-0.5, max(data_0)+1.5,1)
# Histograma para variable_binaria == 0
axes[0].hist(data_0,bins=np.arange(-0.5, max(data_0)+1.5,1),density=True, color='blue', alpha=0.7, edgecolor='black')
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_xlabel('dicrepancy_destcat', fontsize=12)
axes[0].set_ylabel('Relative frequency', fontsize=12)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Histograma para variable_binaria == 1
axes[1].hist(data_1, bins=np.arange(-0.5, max(data_1)+1.5,1), density=True,color='orange', alpha=0.7, edgecolor='black')
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xlabel('dicrepancy_destcat', fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

x_limit = [-1.5,50]
# x_limit = [-1.5,25]
# # # axes[0].set_ylim([0,0.1])
axes[0].set_xlim(x_limit)
axes[1].set_xlim(x_limit)
# axes[1].set_ylim([0,0.1])


# Ajustar espaciado entre los subplots
plt.tight_layout()
plt.show()

In [ ]:
# booleanas

# Contar las frecuencias de variable_booleana para cada valor de variable_binaria
conteo_fraude = base_data[base_data['is_fraud'] == 1]['top_fraud_destination_flag'].value_counts().sort_index()
conteo_no_fraude = base_data[base_data['is_fraud'] == 0]['top_fraud_destination_flag'].value_counts().sort_index()

# Asegurar que ambas categorías tienen valores 0 y 1 (completar si falta alguna categoría)
conteo_fraude = conteo_fraude.reindex([0, 1], fill_value=0)
conteo_no_fraude = conteo_no_fraude.reindex([0, 1], fill_value=0)

# Crear gráficos lado a lado
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Gráfico para No Fraude (variable_binaria = 0) con escala independiente
axes[0].bar([0, 1], conteo_no_fraude, color=['blue', 'orange'], alpha=0.7)
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['No top_fraud_destination_flag', 'top_fraud_destination_flag'])
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico para Fraude (variable_binaria = 1) con escala independiente
axes[1].bar([0, 1], conteo_fraude, color=['blue', 'orange'], alpha=0.7)
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xticks([0, 1])
axes[1].set_xticklabels(['No top_fraud_destination_flag', 'top_fraud_destination_flag'])
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

# Ajustar diseño
plt.tight_layout()
plt.show()

In [ ]:
# booleanas relative

# Contar las frecuencias de variable_booleana para cada valor de variable_binaria
conteo_fraude = base_data[base_data['is_fraud'] == 1]['top_fraud_agency_flag'].value_counts(normalize=True).sort_index()
conteo_no_fraude = base_data[base_data['is_fraud'] == 0]['top_fraud_agency_flag'].value_counts(normalize=True).sort_index()

# Asegurar que ambas categorías tienen valores 0 y 1 (completar si falta alguna categoría)
conteo_fraude = conteo_fraude.reindex([0, 1], fill_value=0)
conteo_no_fraude = conteo_no_fraude.reindex([0, 1], fill_value=0)

# Crear gráficos lado a lado
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Gráfico para No Fraude (variable_binaria = 0) con escala independiente
axes[0].bar([0, 1], conteo_no_fraude, color=['blue', 'orange'], alpha=0.7)
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_ylabel('Relative frequency', fontsize=12)
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['No top_fraud_agency_flag', 'top_fraud_agency_flag'])
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico para Fraude (variable_binaria = 1) con escala independiente
axes[1].bar([0, 1], conteo_fraude, color=['blue', 'orange'], alpha=0.7)
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xticks([0, 1])
axes[1].set_xticklabels(['No top_fraud_destination_flag', 'top_fraud_destination_flag'])
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

# Ajustar diseño
plt.tight_layout()
plt.show()

In [ ]:
# categorical
categorias = sorted(base_data['application'].unique())


data_0 = base_data[base_data['is_fraud'] == 0]['application'].value_counts().reindex(categorias, fill_value=0)
data_1 = base_data[base_data['is_fraud'] == 1]['application'].value_counts().reindex(categorias, fill_value=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=False)  # sharey=False para escalas independientes

axes[0].bar(data_0.index, data_0.values, color='blue', alpha=0.7, edgecolor='black')
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_xlabel('application', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico de barras para variable_binaria == 1
axes[1].bar(data_1.index, data_1.values, color='orange', alpha=0.7, edgecolor='black')
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xlabel('application', fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

# Ajustar espaciado entre subplots
plt.tight_layout()
plt.show()

In [ ]:
# categorical relative
categorias = sorted(base_data['paymenttype'].unique())


data_0 = base_data[base_data['is_fraud'] == 0]['paymenttype'].value_counts(normalize=True).reindex(categorias, fill_value=0)
data_1 = base_data[base_data['is_fraud'] == 1]['paymenttype'].value_counts(normalize=True).reindex(categorias, fill_value=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=False)  # sharey=False para escalas independientes

axes[0].bar(data_0.index, data_0.values, color='blue', alpha=0.7, edgecolor='black')
axes[0].set_title('No Fraud', fontsize=14)
axes[0].set_xlabel('paymenttype', fontsize=12)
axes[0].set_ylabel('Relative Frequency', fontsize=12)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Gráfico de barras para variable_binaria == 1
axes[1].bar(data_1.index, data_1.values, color='orange', alpha=0.7, edgecolor='black')
axes[1].set_title('Fraud', fontsize=14)
axes[1].set_xlabel('paymenttype', fontsize=12)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

# Ajustar espaciado entre subplots
plt.tight_layout()
plt.show()

# Heat Map

In [111]:
encoded_sql = """
SELECT
       *
FROM test_hbg.dbt_jferragut.feature_encoded"""

encoded = pd.read_sql_query(encoded_sql,engine)

In [ ]:
encoded.columns

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = base_data[['lead_time_days',
       'length_of_stay', 'PAYMENTTYPE_BookNowPayLater', 'PAYMENTTYPE_Credit','paymenttype_0',
       'PAYMENTTYPE_PayWithCreditCard', 'top_fraud_agency_flag', 'total_ko_web', 
       'top_fraud_destination_flag', 'adr',
       'dicrepancy_cm', 'dicrepancy_cy', 'dicrepancy_destcat', 'is_fraud']].corr()

# Crear el mapa de calor con Seaborn
plt.figure(figsize=(8, 6))  # Tamaño de la figura
sns.heatmap(correlation_matrix, annot=True, annot_kws={"size":6},cmap='coolwarm', fmt='.2f', linewidths=0.5)

# Personalizar la visualización
plt.title('Correlation Heat Map', fontsize=16)
plt.show()